In [1]:
import os
api_key = os.getenv("OPENAI_API_KEY")
print(api_key)  # 验证是否成功加载

J1O8X0E685L3S8PD


In [2]:
import sys, os
import requests
import pandas as pd

In [3]:
def get_alpha_vantage_data(symbol, function='TIME_SERIES_DAILY', output_size='full'):

    base_url = "https://www.alphavantage.co/query"
    
    params = {
        "function": function,
        "symbol": symbol,
        "outputsize": output_size,
        "apikey": api_key,
        "datatype": "json" 
    }
    
    try:
        response = requests.get(base_url, params=params)
        response.raise_for_status()  
        data = response.json()
        
        if 'Time Series (Daily)' in data:
            time_series = data['Time Series (Daily)']
        elif 'Weekly Time Series' in data:
            time_series = data['Weekly Time Series']
        elif 'Monthly Time Series' in data:
            time_series = data['Monthly Time Series']
        else:
            raise ValueError("无法解析返回的数据格式")
        
        df = pd.DataFrame.from_dict(time_series, orient='index')
        df.index = pd.to_datetime(df.index)  
        df = df.astype(float)  
        
        df.columns = [col.split('. ')[1] if '. ' in col else col for col in df.columns]
        
        df = df.sort_index(ascending=True)
        
        return df
    
    except Exception as e:
        print(f"获取数据时出错: {e}")
        return None

In [4]:
aapl_data = get_alpha_vantage_data('AAPL', function='TIME_SERIES_DAILY', output_size='full')

In [5]:
aapl_data['year'] = aapl_data.index.astype(str).str[:4]
aapl_data = aapl_data[(aapl_data['year'] >= '2015') & (aapl_data['year'] <= '2025')]
aapl_data

,open,high,low,close,volume,year
2015-01-02,111.390,111.44,107.350,109.33,53204626.0,2015
2015-01-05,108.290,108.65,105.410,106.25,64285491.0,2015
2015-01-06,106.540,107.43,104.630,106.26,65797116.0,2015
2015-01-07,107.200,108.20,106.695,107.75,40105934.0,2015
2015-01-08,109.230,112.15,108.700,111.89,59364547.0,2015
...,...,...,...,...,...,...
2025-08-13,231.070,235.00,230.430,233.33,69878546.0,2025
2025-08-14,234.055,235.12,230.850,232.78,51916275.0,2025
2025-08-15,234.000,234.28,229.335,231.59,56038657.0,2025
2025-08-18,231.700,233.12,230.110,230.89,37476188.0,2025


In [6]:
aapl_data.to_csv('C:/Users/Zhangchensi/bootcamp_Chensi_Zhang/project/data/raw/raw_data.csv')